In [2]:
import numpy as np

import pandas as pd
import geopandas as gpd

#import matplotlib.pyplot as plt
#import seaborn as sns

#import folium
import requests

import json


In [1]:
from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.plotting import figure
from bokeh.palettes import brewer
from bokeh.sampledata.sample_geojson import geojson

from bokeh.tile_providers import CARTODBPOSITRON, get_provider

#output_notebook()

In [4]:
data = requests.get('https://data.gov.au/geoserver/vic-suburb-locality-boundaries-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_af33dd8c_0534_4e18_9245_fc64440f742e&outputFormat=json', headers={"Content-Type":"application/json"})
geodata = data.json()

In [5]:
with open('../data/raw/geodata.json', 'w', encoding='utf-8') as f:
    json.dump(geodata, f, indent=4)

In [3]:
gdf = gpd.read_file('../data/raw/geodata.json')
gdf.set_crs('epsg:4283')
# map tile providers provide only a few projections, so casting to that specific projection
# WGS84 (EPSG:4326) or Web Mercator (EPSG:3857) are usually available 
# source: https://kodu.ut.ee/~kmoch/geopython2020/L6/interactive-map-bokeh.html
gdf['geometry'] = gdf['geometry'].to_crs(epsg=3857) 
gdf.head()


,id,lc_ply_pid,dt_create,dt_retire,loc_pid,vic_locali,vic_loca_1,vic_loca_2,vic_loca_3,vic_loca_4,vic_loca_5,vic_loca_6,vic_loca_7,geometry
0,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.1,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"MULTIPOLYGON (((15779040.302 -4173707.184, 157..."
1,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.2,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"MULTIPOLYGON (((16549732.035 -4494406.666, 165..."
2,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.3,6672,2011-08-31,None,VIC2862,2012-04-27,None,WOORNDOO,None,None,G,None,2,"MULTIPOLYGON (((15910102.922 -4576437.989, 159..."
3,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.4,6673,2011-08-31,None,VIC734,2018-08-03,None,DEPTFORD,None,None,G,None,2,"MULTIPOLYGON (((16455621.450 -4531505.374, 164..."
4,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.5,6674,2011-08-31,None,VIC2900,2012-04-27,None,YANAC,None,None,G,None,2,"MULTIPOLYGON (((15727194.252 -4300425.394, 157..."


In [4]:
gdf['color'] = 'red'
gdf.head()

,id,lc_ply_pid,dt_create,dt_retire,loc_pid,vic_locali,vic_loca_1,vic_loca_2,vic_loca_3,vic_loca_4,vic_loca_5,vic_loca_6,vic_loca_7,geometry,color
0,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.1,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"MULTIPOLYGON (((15779040.302 -4173707.184, 157...",red
1,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.2,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"MULTIPOLYGON (((16549732.035 -4494406.666, 165...",red
2,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.3,6672,2011-08-31,None,VIC2862,2012-04-27,None,WOORNDOO,None,None,G,None,2,"MULTIPOLYGON (((15910102.922 -4576437.989, 159...",red
3,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.4,6673,2011-08-31,None,VIC734,2018-08-03,None,DEPTFORD,None,None,G,None,2,"MULTIPOLYGON (((16455621.450 -4531505.374, 164...",red
4,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.5,6674,2011-08-31,None,VIC2900,2012-04-27,None,YANAC,None,None,G,None,2,"MULTIPOLYGON (((15727194.252 -4300425.394, 157...",red


In [5]:
#gdf['geometry'] = gdf['geometry'].simplify(0.05, preserve_topology=True) # if we want to reduce precision of each polygon to make it quicker to display, 0.05 is the reducing coeficitient or whateve

In [12]:
output_file("../plots/test.html", title="Testing suburb plotting")

geo_source = GeoJSONDataSource(geojson=gdf.to_json())

tile_provider = get_provider(CARTODBPOSITRON)

# range bounds supplied in web mercator coordinates
p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)
# crs "name": "urn:ogc:def:crs:EPSG::4283"


# Add patch renderer to figure.
states = p.patches('xs', 'ys', source=geo_source,
                   fill_color={'field': 'color'},   #here comes the transform function
                   line_color='gray',
                   line_width=0.25,
                   fill_alpha=0.2)

p.add_tools(HoverTool(renderers=[states],
                      tooltips=[('Suburb', '@vic_loca_2'), ('Color', '@color')]))

show(p);


Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "file:///home/toomas/ADS/generic-real-estate-consulting-project ...
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 
